# Shortest Route: Finding the most efficient driving routes between New York City landmarks

## Description of Approach

##### **Approach**
- We selected NYC because it's a group member's former home
- Nodes were defined by selection iconic NYC locations (ex. Times Square, Rockefeller Center, Empire State Building, etc.)
- The maps in this notebook were built using `folium`
- For both base and rush hour scenarios, we used Google Maps data. The dataset included information on peak travel times, but we opted to use a model instead to show increase congestion. The model applies a multiple between 1 and 3 to each edge.

##### **Assumptions**
- The time from node A to node B is the same as the time from node B to node A

## Graph Structure & Algorithm Implementation

We chose to represent our graph with ........

Dijkstra's Algorithm was implemented by ........

Design choices include setting the `folium` map to highlight the shortest path in red, and making the backgroup map of NYC more simplistic/subtle to increase contrast

## Code-Based Verification

### Import Libraries, Functions, Data

In [1]:
import sys
import os
import json
import folium
import random
import numpy as np

project_root = os.path.abspath(os.path.join('..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from modules.graph import Graph
from modules.mapGraph import build_graph_from_json
from modules.displayMap import build_graph_mappings
from modules.simulate import sim_traffic_data

In [3]:
with open('manhattan_driving_distances.json', 'r') as f:
    loaded_dict = json.load(f)

### Base Case: Average Traffic Conditions

In [4]:
start_node = "Battery Park / South Ferry"
end_node = "Rockefeller Center"

#### Without Uncertainty

In [5]:
metric = "avg"
distances = sim_traffic_data(loaded_dict, start_node, metric, False)[1]
g = sim_traffic_data(loaded_dict, start_node, metric, False)[0]

print(f"Average Travel Time from \033[1m{start_node}\033[0m to \033[1m{end_node}\033[0m: {round(distances[1],2)} min")

build_graph_mappings(loaded_dict, start_node, end_node, metric, False)

Average Travel Time from Battery Park / South Ferry to Rockefeller Center: 30.53 min
Shortest path names: ['Battery Park / South Ferry', 'Washington Square Park', 'Union Square', 'Flatiron / Madison Sq Park', 'Empire State Building', 'Bryant Park / NYPL', 'Rockefeller Center']


#### With Uncertainty

In [6]:
distances = sim_traffic_data(loaded_dict, start_node, metric, True)[1]
g = sim_traffic_data(loaded_dict, start_node, metric, True)[0]

print(f"Average Travel Time from \033[1m{start_node}\033[0m to \033[1m{end_node}\033[0m: {round(distances[1],2)} min")

if round(distances[1],2) == float('inf'):
    print(f"Road Closure: Unable to drive from {start_node} to {end_node}")
else:
    build_graph_mappings(loaded_dict, start_node, end_node, metric, True)

Average Travel Time from Battery Park / South Ferry to Rockefeller Center: 117.24 min
Shortest path names: ['Battery Park / South Ferry', 'Washington Square Park', 'Chelsea Market', 'Union Square', 'Flatiron / Madison Sq Park', 'Empire State Building', 'Bryant Park / NYPL', 'Rockefeller Center']


### Rush Hour Scenario: Peak Traffic Conditions

#### Without Uncertainty

In [7]:
start_node = "Battery Park / South Ferry"
end_node = "Rockefeller Center"
metric = "peak"
distances = sim_traffic_data(loaded_dict, start_node, metric, False)[1]
g = sim_traffic_data(loaded_dict, start_node, metric, False)[0]

print(f"Average Travel Time from \033[1m{start_node}\033[0m to \033[1m{end_node}\033[0m: {round(distances[1],2)} min")

build_graph_mappings(loaded_dict, start_node, end_node, metric, False)

Average Travel Time from Battery Park / South Ferry to Rockefeller Center: 66.23 min
Shortest path names: ['Battery Park / South Ferry', 'Washington Square Park', 'Union Square', 'Flatiron / Madison Sq Park', 'Empire State Building', 'Grand Central Terminal', 'Rockefeller Center']


#### With Uncertainty

In [8]:
distances = sim_traffic_data(loaded_dict, start_node, metric, True)[1]
g = sim_traffic_data(loaded_dict, start_node, metric, True)[0]

print(f"Average Travel Time from \033[1m{start_node}\033[0m to \033[1m{end_node}\033[0m: {round(distances[1],2)} min")

if round(distances[1],2) == float('inf'):
    print(f"Road Closure: Unable to drive from {start_node} to {end_node}")
else:
    build_graph_mappings(loaded_dict, start_node, end_node, metric, True)

Average Travel Time from Battery Park / South Ferry to Rockefeller Center: 116.77 min
Shortest path names: ['Battery Park / South Ferry', 'Washington Square Park', 'Union Square', 'Flatiron / Madison Sq Park', 'Empire State Building', 'Grand Central Terminal', 'Rockefeller Center']


## Discussion & Analysis

##### **Route**
Due to the shape of the graph (layout of selected NYC landmarks), the shortest path from Battery Park to Rockefeller Center is almost always the same despite the scenario <br>
- Battery Park / South Ferry $\rightarrow$ Washington Square Park $\rightarrow$ Union Square $\rightarrow$ Flatiron / Madison Sq Park $\rightarrow$ Empire State Building $\rightarrow$ Grand Central Terminal $\rightarrow$ Rockefeller Center <br>

However, road closures can cause a deviation in the path. For example, when we added uncertainty to the average traffic scenario, the route between Union Square and Washington Square park was blocked off. As a result, the shortest path diverted to Chelsea Market before continuing as usual.

##### **Uncertainty**

Uncertainty can lead to a large increase in travel time. For example, at peak traffic (without uncertainty) the time between Battery Park and Rockefeller Center was 66.23 minutes. After factoring in uncertainty, the time increased to 116.77 minutes (+76%)! Also, as noted above, road closures due to uncertainty can change the shortest path as well as add travel time.

##### **Interpreting Results**

##### **Assumptions & Limitations**
Our results are conditioned on the initial selection of nodes. There are of course many different ways to travel to each of the NYC landmarks; a road closure would not actually mean you can't make it to a certain destination. We also assumed that travel time from node A to node B was the same as for node B to node A. In reality this would not be true given NYC's many one-way streets and varying traffic patterns.